# Paradigmas y Programación para Ciencia de Datos
## Programa 4


Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

Carlo Kiliano Ferrera Guadarrama, Aldo Sebastián Altamirano Vázquez, Edgar Omar Cruz Guitierrez, Leonardo Negrete Mancera




#Numero Decimal a Cadena Binaria

In [ ]:
def dec2float(decimal_number):
    """
    Convierte un número decimal a su representación binaria IEEE 754
    de simple precisión (32 bits). Asume números 'normales' para simplificar.
    """

    # --- 1. Signo ---
    if decimal_number < 0:
        sign_bit = '1'
        num = abs(decimal_number)
    else:
        sign_bit = '0'
        num = decimal_number

    # --- 2. Conversión a binario (Entera y Fraccionaria) ---
    entera = int(num)
    fraccion = num - entera

    if entera == 0:
        entera_bin = '0'
    else:
        entera_bin = bin(entera).replace('0b', '')

    fraccion_bin = ''
    MAX_FRAC_BITS = 30
    temp_frac = fraccion
    for _ in range(MAX_FRAC_BITS):
        temp_frac *= 2
        bit = int(temp_frac)
        fraccion_bin += str(bit)
        temp_frac -= bit
        if temp_frac == 0:
            break

    # --- 3. Normalización (Determinar E y Mantisa) ---

    # Manejo de cero (simplificado)
    if num == 0:
        return "0" * 32

    if entera_bin != '0':
        # Caso: Número >= 1.0. El punto se mueve a la IZQUIERDA.
        E = len(entera_bin) - 1
        normalized_fraction = entera_bin[1:] + fraccion_bin

    else: # Caso: Número < 1.0. El punto se mueve a la DERECHA.
        first_one_index = fraccion_bin.find('1')

        # Si no hay '1's, es cero, ya manejado arriba, pero se incluye por robustez de la lógica < 1.0
        if first_one_index == -1:
             return "0" * 32

        E = -(first_one_index + 1)
        normalized_fraction = fraccion_bin[first_one_index + 1:]

    # --- 4. Exponente Sesgado (Biased Exponent) ---
    BIAS = 127
    exponent_biased = E + BIAS
    exponent_bin = bin(exponent_biased).replace('0b', '').zfill(8)

    # --- 5. Mantisa (Fracción) ---
    mantissa_bits = normalized_fraction[:23].ljust(23, '0')

    # --- 6. Concatenar y devolver ---
    ieee_754 = sign_bit + exponent_bin + mantissa_bits

    return ieee_754

# --- Prueba con el ejemplo de la imagen ---
number_to_convert = 19.765625
result = dec2float(number_to_convert)
print(f"dec2float({number_to_convert})")
print(f"Devuelve:\n'{result}'")

# El resultado esperado es: '01000001100111100010000000000000'

dec2float(19.765625)
Devuelve:
'01000001100111100010000000000000'


#Cadena Binaria a Valor Decimal

In [ ]:
def float2dec(binary_string):
    """
    Convierte una cadena binaria IEEE 754 de simple precisión (32 bits)
    a su valor decimal, siguiendo estrictamente la fórmula de la imagen:
    d = (-1)^S * 2^(E-127) * [1 + sum(b_i * 2^-i)]

    :param binary_string: Cadena de 32 caracteres '0's y '1's.
    :returns: El número decimal representado.
    """

    if len(binary_string) != 32:
        return "Error: La cadena binaria debe tener exactamente 32 bits."

    # --- 1. Decodificar los campos ---

    # a. Signo (S): bit 0
    S = int(binary_string[0])

    # b. Exponente Sesgado (E): bits 1 al 8 (8 bits)
    exponent_bin = binary_string[1:9]
    E = int(exponent_bin, 2)

    # c. Mantisa (b_i): bits 9 al 31 (23 bits)
    mantissa_bin = binary_string[9:]

    # --- 2. Calcular el Factor de Signo y Exponente ---

    # Factor de Signo: (-1)^S
    sign_factor = (-1)**S

    # Factor de Exponente: 2^(E - 127)
    exponent_factor = 2**(E - 127)

    # --- 3. Calcular el Factor de Significando (la suma) ---

    # Factor de Significando: [1 + sum(b_i * 2^-i)]
    sum_mantissa_term = 0.0

    # La suma va de i=1 hasta i=23
    for i in range(23):
        b_i = int(mantissa_bin[i]) # b_i es el bit de la mantisa en la posición i
        # El índice i en Python (0 a 22) corresponde a i+1 en la fórmula (1 a 23)
        # El exponente es -(i+1)
        if b_i == 1:
             sum_mantissa_term += (b_i * (2**(-(i + 1))))

    significand_factor = 1.0 + sum_mantissa_term

    # --- 4. Calcular el Valor Final (d) ---

    decimal_value = sign_factor * exponent_factor * significand_factor

    return decimal_value

# --- Prueba con el ejemplo de la imagen ---
binary_input = '01000001100111100010000000000000'

result = float2dec(binary_input)
print(f"float2dec('{binary_input}')")
print(f"Devuelve:\n{result}")

# Comprobación del resultado
# El resultado es 19.765625, exactamente como se muestra en tu apunte.

float2dec('01000001100111100010000000000000')
Devuelve:
19.765625
